In [ ]:
# default_exp targetdata

# targetdata

> Get underlying data to calculate Rule Set 3 target scores


Get amino acid sequences and protein domain information

In [ ]:
# export
import requests
import json
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm
import warnings
import os
from scipy import stats
import multiprocessing

In [ ]:
design_df = pd.read_table('test_data/sgrna-designs.txt')
max_n_jobs = multiprocessing.cpu_count()

## Get amino acid sequences

In [ ]:
# export
def ensembl_post(ext, data, headers=None, params=None):
    """Generic wrapper for using POST requests to the ensembl rest API

    :param ext: str, url extension
    :param data: dict, query data
    :param headers: dict or None,  meta-information for query
    :param params: dict or None, parameters for query
    :return: Response object
    """
    if params is None:
        params = {}
    if headers is None:
        headers = {}
    data = json.dumps(data)
    r = requests.post("https://rest.ensembl.org"+ext, headers=headers, data=data, params=params)
    if not r.ok:
        r.raise_for_status()
    else:
        return r

In [ ]:
# export
def chunks(lst, n):
    """Yield successive n-sized chunks from lst.

    lst: list
    n: int

    returns: generator of list chunks
    """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def post_transcript_sequence_chunk(ids, params, headers):
    """Helper function for post_transcript_sequence

    :param ids: list
    :param params: dict
    :param headers: dict
    :return: dict
    """
    data = {'ids': ids}
    r = ensembl_post("/sequence/id/", data=data, params=params,
                     headers=headers)
    seq = r.json()
    return seq

def post_transcript_sequence(ensembl_ids, seq_type='protein', max_queries=50,
                             n_jobs=1, **kwargs):
    """Request multiple types of sequence by stable identifier. Supports feature masking and expand options.
    Uses https://rest.ensembl.org/documentation/info/sequence_id_post

    :param ensembl_ids: list of str
    :param seq_type: str, one of [genomic, cds, cdna, protein]
    :param max_queries: int, maximum number of queries for post
    :param n_jobs: int, number of jobs to run in parallel
    :param kwargs: additional parameter arguments
    :return: list, dict of sequences 5' to 3' in the same orientation as the input transcript
    """
    headers={"content-type" : "application/json", "accept" : "application/json"}
    params = {'type': seq_type, **kwargs}
    id_chunks = list(chunks(ensembl_ids, max_queries))
    seqs = Parallel(n_jobs=n_jobs)(delayed(post_transcript_sequence_chunk)
                                   (ids, params, headers) for ids in tqdm(id_chunks))
    # flatten list
    seqs = [item for sublist in seqs for item in sublist]
    return seqs

In [ ]:
assert(post_transcript_sequence(["ENSG00000157764", "ENSG00000248378"],
                                seq_type='genomic')[0]['seq'][:4] == 'CTTC')

100%|██████████| 1/1 [00:01<00:00,  1.72s/it]


In [ ]:
# export
def build_transcript_aa_seq_df(design_df, transcript_id_col='Target Transcript',
                               transcript_len_col='Target Total Length', n_jobs=1):
    """Get amino acid sequence for transcripts of interest

    :param design_df: DataFrame
    :param transcript_id_col: str, column with ensembl transcript id
    :param transcript_len_col: str, column with length of transcript
    :param n_jobs: int, number of jobs to use to query transcripts
    :return: DataFrame
    """
    unique_transcripts = design_df[[transcript_id_col, transcript_len_col]].drop_duplicates()
    unique_transcripts['Transcript Base'] = unique_transcripts[transcript_id_col].str.split('.', expand=True)[0]
    print("Getting amino acid sequences")
    aa_seqs = post_transcript_sequence(unique_transcripts['Transcript Base'].to_list(),
                                       n_jobs=n_jobs)
    aa_seq_df = (pd.DataFrame(aa_seqs)
                 .rename({'query': 'Transcript Base'}, axis=1))
    missing_seqs = (unique_transcripts['Transcript Base'][~unique_transcripts['Transcript Base'].isin(
        aa_seq_df['Transcript Base']
    )])
    if len(missing_seqs) > 0:
        warnings.warn('Unable to find translations for the following transcripts: ' + ', '.join(missing_seqs))
    aa_seq_len_df = (unique_transcripts.merge(aa_seq_df, on='Transcript Base'))
    aa_seq_len_df['AA len'] = aa_seq_len_df['seq'].str.len()
    filtered_aa_seq_len_df = (aa_seq_len_df[aa_seq_len_df[transcript_len_col] ==
                                            (aa_seq_len_df['AA len'] + 1)*3 ]
                              .reset_index(drop=True))
    filtered_seqs = (aa_seq_len_df['Transcript Base'][~aa_seq_len_df['Transcript Base'].isin(
        filtered_aa_seq_len_df['Transcript Base']
    )])
    if len(filtered_seqs) > 0:
        warnings.warn('Filtered transcripts where the transcript length and amino acid ' +
                      'sequence length did not agree: ' + ', '.join(filtered_seqs))
    return filtered_aa_seq_len_df

In [ ]:
transcript_aa_seq_df = build_transcript_aa_seq_df(design_df, n_jobs=2)
transcript_aa_seq_df

Getting amino acid sequences


100%|██████████| 4/4 [00:02<00:00,  1.47it/s]


,Target Transcript,Target Total Length,Transcript Base,version,desc,molecule,seq,id,AA len
0,ENST00000259457.8,834,ENST00000259457,3,None,protein,MAAVSVYAPPVGGFSFDNCRRNAVLEADFAKRGYKLPKVRKTGTTI...,ENSP00000259457,277
1,ENST00000394249.8,1863,ENST00000394249,3,None,protein,MRRSEVLAEESIVCLQKALNHLREIWELIGIPEDQRLQRTEVVKKH...,ENSP00000377793,620
2,ENST00000361337.3,2298,ENST00000361337,2,None,protein,MSGDHLHNDSQIEADFRLNDSHKHKDKHKDREHRHKEHKKEKDREK...,ENSP00000354522,765
3,ENST00000368328.5,267,ENST00000368328,4,None,protein,MALSTIVSQRKQIKRKAPRGFLKRVFKRKKPQLRLEKSGDLLVHLN...,ENSP00000357311,88
4,ENST00000610426.5,783,ENST00000610426,1,None,protein,MPQNEYIELHRKRYGYRLDYHEKKRKKESREAHERSKKAKKMIGLK...,ENSP00000483484,260
...,...,...,...,...,...,...,...,...,...
195,ENST00000339374.11,1371,ENST00000339374,6,None,protein,MAAALKCLLTLGRWCPGLGVAPQARALAALVPGVTQVDNKSGFLQK...,ENSP00000343041,456
196,ENST00000393047.8,882,ENST00000393047,3,None,protein,MSRIPLGKVLLRNVIRHTDAHNKIQEESDMWKIRELEKQMEDAYRG...,ENSP00000376767,293
197,ENST00000454402.7,1023,ENST00000454402,2,None,protein,METSALKQQEQPAATKIRNLPWVEKYRPQTLNDLISHQDILSTIQK...,ENSP00000408295,340
198,ENST00000254998.3,423,ENST00000254998,2,None,protein,MASVDFKTYVDQACRAAEEFVNVYYTTMDKRRRLLSRLYMGTATLV...,ENSP00000254998,140


## Get protein domains

In [ ]:
# export
def ensembl_get(ext, query=None, headers=None, params=None):
    """Generic wrapper for using GET requests to the ensembl rest API

    ext: str, url extension |
    query: str or None, end of url extension specifying species, taxon, esnembl_id etc |
    headers: dict or None,  meta-information for query |
    params: dict or None, parameters for query |

    returns: Response object
    """
    if query is None:
        query = ''
    if params is None:
        params = {}
    if headers is None:
        headers = {}
    r = requests.get("https://rest.ensembl.org"+ext+query, params=params, headers=headers)
    if not r.ok:
        r.raise_for_status()
    else:
        return r

def get_translation_overlap(ensembl_id):
    """Get features that overlap with translation, such as protein domains

    :param ensembl_id: str
    :return: DataFrame
    """
    headers = {'content-type': 'application/json'}
    ext = '/overlap/translation/' + ensembl_id
    r = ensembl_get(ext, headers=headers)
    decoded = r.json()
    return decoded

In [ ]:
brca1_overlap = get_translation_overlap('ENSP00000350283')
assert 'BRCT domain' in pd.DataFrame(brca1_overlap)['description'].to_list()

In [ ]:
# export
def build_translation_overlap_df(protein_ids, n_jobs=1):
    """Get protein domain information

    :param protein_ids: list of str, ensemble protein IDs
    :param n_jobs: int
    :return: DataFrame
    """
    print('Getting protein domains')
    translation_overlap_list = Parallel(n_jobs=n_jobs)(delayed(get_translation_overlap)
                                                       (id) for id in tqdm(protein_ids))
    # flatten list
    translation_overlap_list = [item for sublist in translation_overlap_list for item in sublist]
    translation_overlap_df = pd.DataFrame(translation_overlap_list).rename({'Parent': 'Transcript Base'}, axis=1)
    return translation_overlap_df

In [ ]:
translation_overlap_df = build_translation_overlap_df(transcript_aa_seq_df['id'],
                                                      n_jobs=2)
translation_overlap_df

Getting protein domains


100%|██████████| 200/200 [00:48<00:00,  4.09it/s]


,hseqname,translation_id,interpro,start,Transcript Base,align_type,feature_type,cigar_string,id,type,hit_end,end,hit_start,seq_region_name,description
0,3.60.20.10,976188,IPR029055,44,ENST00000259457,None,protein_feature,,3.60.20.10,Gene3D,233,277,1,ENSP00000259457,"Nucleophile aminohydrolases, N-terminal"
1,PF12465,976188,IPR024689,235,ENST00000259457,None,protein_feature,,PF12465,Pfam,36,271,1,ENSP00000259457,"Proteasome beta subunit, C-terminal"
2,PF00227,976188,IPR001353,41,ENST00000259457,None,protein_feature,,PF00227,Pfam,190,221,2,ENSP00000259457,"Proteasome, subunit alpha/beta"
3,PR00141,976188,IPR000243,51,ENST00000259457,None,protein_feature,,PR00141,PRINTS,0,66,0,ENSP00000259457,"Peptidase T1A, proteasome beta-subunit"
4,PR00141,976188,IPR000243,171,ENST00000259457,None,protein_feature,,PR00141,PRINTS,0,182,0,ENSP00000259457,"Peptidase T1A, proteasome beta-subunit"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,Coil,975385,,516,ENST00000381685,None,protein_feature,,Coil,ncoils,0,537,0,ENSP00000371101,
5276,Coil,975385,,558,ENST00000381685,None,protein_feature,,Coil,ncoils,0,587,0,ENSP00000371101,
5277,Coil,975385,,640,ENST00000381685,None,protein_feature,,Coil,ncoils,0,669,0,ENSP00000371101,
5278,PTHR14927,975385,IPR040382,1,ENST00000381685,None,protein_feature,,PTHR14927,PANTHER,676,682,5,ENSP00000371101,Nucleolar protein 10/Enp2


## Store data using parquet for later use

In [ ]:
# export
def write_transcript_data(design_df, transcript_id_col='Target Transcript',
                          transcript_len_col='Target Total Length', n_jobs=1,
                          overwrite=True, filepath='./data/target_data/',
                          aa_seq_name='aa_seqs.pq',
                          protein_domain_name='protein_domains.pq'):
    """Write amino acid sequences and protein domain information to parquet files

    :param design_df: DataFrame
    :param transcript_id_col: str
    :param transcript_len_col: str
    :param n_jobs: int
    :param overwrite: bool, whether to overwrite existing file
    :param filepath: str, directory for output sequences
    :param aa_seq_name: str, name of amino acid sequence file
    :param protein_domain_name: str, name of protein domain file
    """
    if (os.path.isfile(filepath + aa_seq_name) or os.path.isfile(filepath + protein_domain_name)) and (not overwrite):
        raise ValueError('Transcript data already exits and cannot be overwritten')
    else:
        transcript_aa_seq_df = build_transcript_aa_seq_df(design_df, transcript_id_col=transcript_id_col,
                                                          transcript_len_col=transcript_len_col,
                                                          n_jobs=n_jobs)
        translation_overlap_df = build_translation_overlap_df(transcript_aa_seq_df['id'],
                                                              n_jobs=n_jobs)
        if not os.path.isdir(filepath):
            print('Creating new directory ' + filepath)
            os.makedirs(filepath)
        transcript_aa_seq_df.to_parquet(path=filepath + aa_seq_name, engine='pyarrow',
                                        index=False)
        translation_overlap_df.to_parquet(path=filepath + protein_domain_name, engine='pyarrow',
                                          index=False)

In [ ]:
write_transcript_data(design_df, n_jobs=2)
assert os.path.isfile('./data/target_data/' + 'aa_seqs.pq')

Getting amino acid sequences


100%|██████████| 4/4 [00:02<00:00,  1.42it/s]


Getting protein domains


100%|██████████| 200/200 [00:48<00:00,  4.13it/s]


# Get Conservation Scores

Get PhyloP conservation scores

In [ ]:
# export
def get_transcript_info(base_transcript):
    """Using an ensembl transcript ID, get

    :param base_transcript: str
    :return: (exon_df, trans_sr, chr)
        exon_df: DataFrame, with global exon start and end position
        trans_sr: Series, with global translation start and stop positions for CDS and translation length
        chr: str

    """
    r = ensembl_get("/lookup/id/" + base_transcript + "?expand=1",
                    headers={"Content-Type": "application/json"}, params={'expand': '1'})
    decoded = r.json()
    exon_df = pd.DataFrame(decoded['Exon'])
    trans_sr = pd.Series(decoded['Translation'])
    chr = decoded['seq_region_name']
    return exon_df, trans_sr, chr

In [ ]:
exon_df, trans_sr, chr = get_transcript_info('ENST00000259457')
assert chr == '9'
assert trans_sr['length'] == 277 # number of aas
assert exon_df.shape[0] == 8 # eight exons

In [ ]:
# export
def get_conservation(chr, start, end, genome):
    """Get conservation scores for a given region of a genome

    :param chr: str, chromosome number
    :param start: int
    :param end: int
    :param genome: str
    :return: DataFrame
    """
    api_url = 'http://api.genome.ucsc.edu/getData/track'
    if genome == 'hg38':
        track = 'phyloP100way'
    elif genome == 'mm39':
        track = 'phyloP35way'
    else:
        raise ValueError('Genome not recognized')
    chrom = 'chr' + chr
    params = {
        'genome': genome,
        'track': track,
        'start': start,
        'end': end,
        'chrom': chrom
    }
    results = requests.get(api_url, data=params)
    if results.ok:
        value_df = (pd.DataFrame([pd.Series(x) for x in pd.read_json(results.content.decode('utf8'))[chrom].values])
                    .rename(columns={'value': 'conservation'}))
    else:
        raise ValueError(results.reason)
    return value_df

In [ ]:
rps20_exon_conservation = get_conservation('8', 56074060, 56074159, 'hg38')
cd274_exon_conservation = get_conservation('9', 5457079, 5457420, 'hg38')
assert rps20_exon_conservation['conservation'].mean() > cd274_exon_conservation['conservation'].mean()

In [ ]:
# export
def get_exon_conservation(exon_df, chr, genome):
    """Get conservation scores for each exon

    :param exon_df: DataFrame
    :param chr: str
    :param genome: str
    :return: DataFrame
    """
    conservation_dict = {}
    for i, row in exon_df.set_index('id').iterrows():
        # subtract one since the nucleotide conservation corresponds to the "end" index
        conservation_dict[i] = get_conservation(chr, row['start'] - 1, row['end'], genome)
        # get the conservation of i
    conservation_df = (pd.concat(conservation_dict)
                       .reset_index(level=0)
                       .reset_index(drop=True)
                       .rename({'level_0': 'exon_id',
                                'end': 'genomic position'}, axis=1)
                       .drop('start', axis=1))
    return conservation_df


def get_transcript_conservation(transcript_id, target_strand, genome):
    """Get conservation scores for a transcript

    :param transcript_id: str
    :param target_strand: str, '+' or '-'
    :param genome: str
    :return: DataFrame
    """
    exon_df, trans_sr, chr = get_transcript_info(transcript_id)
    # only include translated positions
    exon_df['start'] = exon_df['start'].apply(lambda x: max(x, trans_sr['start']))
    exon_df['end'] = exon_df['end'].apply(lambda x: min(x, trans_sr['end']))
    exon_df = exon_df[exon_df['end'] > exon_df['start']].reset_index(drop=True)
    conservation_df = get_exon_conservation(exon_df, chr, genome)
    conservation_df['Transcript Base'] = transcript_id
    if target_strand == '-':
        ascending = False
    else:
        ascending = True
    conservation_df = (conservation_df
                       .sort_values('genomic position', ascending=ascending)
                       .reset_index(drop=True))
    conservation_df['target position'] = conservation_df.index + 1
    conservation_df['chromosome'] = chr
    conservation_df['genome'] = genome
    conservation_df['translation length'] = trans_sr['length']
    return conservation_df

In [ ]:
rps20_conservation_df = get_transcript_conservation('ENST00000009589', '-', 'hg38')
cd274_conservation_df = get_transcript_conservation('ENST00000381577', '+', 'hg38')
assert rps20_conservation_df['conservation'].mean() > cd274_conservation_df['conservation'].mean()

In [ ]:
# export
def get_transcript_conservation_safe(transcript_id, target_strand, genome):
    """Helper function for parrallel query. Return None when conservation dataframe cannot be assembled"""
    try:
        return get_transcript_conservation(transcript_id, target_strand, genome)
    except:
        return None


def build_conservation_df(design_df, n_jobs=1):
    transcript_refseq_df = (design_df[['Target Transcript', 'Strand of Target', 'Target Total Length']]
                            .drop_duplicates())
    if not (transcript_refseq_df['Target Transcript'].str.startswith('ENST') |
            transcript_refseq_df['Target Transcript'].str.startswith('ENSMUST')).all():
        raise ValueError('Must supply human or mouse Ensembl transcript IDs as input')
    print('Getting conservation')
    transcript_refseq_df['Transcript Base'] = (transcript_refseq_df['Target Transcript'].str.split('.', expand=True)[0])
    transcript_refseq_df['genome'] = transcript_refseq_df['Transcript Base'].apply(lambda trans:
                                                                                   'mm39' if 'MUS' in trans else 'hg38')
    all_transcript_conservation_list = Parallel(n_jobs)(delayed(get_transcript_conservation_safe)
                                                        (row['Transcript Base'],
                                                         row['Strand of Target'],
                                                         row['genome'])
                                                         for _, row in tqdm(transcript_refseq_df.iterrows(),
                                                                            total=transcript_refseq_df.shape[0]))
    transcript_conservation_list = []
    failed_list = []
    transcript_list = transcript_refseq_df['Transcript Base'].to_list()
    for i, conservation_df in enumerate(all_transcript_conservation_list):
        if conservation_df is None:
            failed_list.append(transcript_list[i])
        else:
            transcript_conservation_list.append(conservation_df)
    if len(failed_list) > 0:
        warnings.warn('Failed to get conservation scores for ' + str(len(failed_list)) +
                      ' transcripts' + ', '.join(failed_list))
    transcript_conservation_df = (pd.concat(transcript_conservation_list))
    transcript_cons_designs = (transcript_conservation_df
                               .merge(transcript_refseq_df, how='inner',
                                      on=['Transcript Base', 'genome']))
    filtered_transcript_conservation = transcript_cons_designs[
        (transcript_cons_designs['translation length'] + 1)*3 == transcript_cons_designs['Target Total Length']].copy()
    mismatched_transcripts = transcript_conservation_df['Transcript Base'][
        ~transcript_conservation_df['Transcript Base'].isin(filtered_transcript_conservation['Transcript Base'])]
    if len(mismatched_transcripts) > 0:
        warnings.warn('Filtered: ' + str(len(mismatched_transcripts)) +
                      ' transcripts with mismatched length:' + ','.join(mismatched_transcripts))
    filtered_transcript_conservation['ranked_conservation'] = (filtered_transcript_conservation.groupby('Transcript Base')
                                                               ['conservation']
                                                               .rank(pct=True))
    return filtered_transcript_conservation

In [ ]:
conservation_df = build_conservation_df(design_df, n_jobs=max_n_jobs)
assert (conservation_df.groupby('Target Transcript')
        .apply(lambda df: stats.spearmanr(df['conservation'], df['ranked_conservation'])[0])
        > 0.99).all()
assert (conservation_df.loc[(conservation_df['Transcript Base'] == 'ENST00000361337') &
                            (conservation_df['target position'].between(432*3, 663*3)),
                            'ranked_conservation'].mean() >  # pfam
        conservation_df.loc[(conservation_df['Transcript Base'] == 'ENST00000361337') &
                            (conservation_df['target position'].between(36*3, 199*3)),  # mobidblit
                            'ranked_conservation'].mean())

Getting conservation


100%|██████████| 200/200 [03:33<00:00,  1.07s/it]


In [ ]:
# export
def write_conservation_data(design_df, n_jobs=1,
                            overwrite=True, filepath='./data/target_data/',
                            cons_file_name='conservation.pq'):
    """Write conservation scores to parquet files

    :param design_df: DataFrame
    :param n_jobs: int
    :param overwrite: bool, whether to overwrite existing file
    :param filepath: str, directory for output sequences
    :param cons_file_name: str, name of conservation file
    """
    if os.path.isfile(filepath + cons_file_name) and (not overwrite):
        raise ValueError('Conservation data already exits and cannot be overwritten')
    else:
        conservation_df = build_conservation_df(design_df, n_jobs=n_jobs)
        if not os.path.isdir(filepath):
            print('Creating new directory ' + filepath)
            os.makedirs(filepath)
        conservation_df.to_parquet(path=filepath + cons_file_name, engine='pyarrow',
                                   index=False)

In [ ]:
write_conservation_data(design_df, n_jobs=max_n_jobs)
assert os.path.isfile('./data/target_data/' + 'conservation.pq')

Getting conservation


100%|██████████| 200/200 [05:16<00:00,  1.58s/it]
